Kendra Robbins

# Iterative Solvers lab

In [12]:
import numpy as np
from numpy import linalg as LA
import scipy
from scipy import linalg as la
from matplotlib import pyplot as plt
import pandas as pd
import sympy as sy
import math

#### Problem 1

In [42]:
def jacobi(A,b,tol=1e-6,maxiters=500):
    
    #D will be a diagonal matrix with diagonal entries that are the inverses of A's diagonal entries
    D_inv = np.zeros_like(A)
    
    for j in range(np.shape(A)[0]):
        D_inv[j,j]= 1/A[j,j]
        
    dist = 1
    k=0
    x=np.zeros(np.shape(A)[0])
    
    while k<maxiters and dist>tol:
        xnext = x**k + D_inv@(b-A@x)
        dist = la.norm(x-xnext, np.inf)
        x= xnext 
        k =+1
    
    if k==maxiters:
        print("The function did not converge")
        
    else: 
        return x 
    

In [43]:
def diag_dom(n, num_entries=None):
    if num_entries is None:
        num_entries = int(n**1.5) - n
    A = np.zeros((n,n))
    rows = np.random.choice(np.arange(0,n), size=num_entries)
    cols = np.random.choice(np.arange(0,n), size=num_entries)
    data = np.random.randint(-4, 4, size=num_entries)
    for i in range(num_entries):
        A[rows[i], cols[i]] = data[i]
    for i in range(n):
        A[i,i] = np.sum(np.abs(A[i])) + 1
    return A

In [45]:
b = np.random.random(4)
A=diag_dom(4)

C=jacobi(A,b)

In [46]:
np.allclose(b,A@C)

True